In [1]:
from scipy.io import arff
import pandas as pd
data = arff.loadarff('seismic-bumps.arff')
dataset = pd.DataFrame(data[0])
datainfo = pd.DataFrame(data[1])
# print(dataset)
# print(datainfo)


In [2]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy
# from sklearn.model_selection import train_test_split
# integer encode
label_encoder = LabelEncoder()
dataset.iloc[:,0] = label_encoder.fit_transform(dataset.iloc[:,0])
dataset.iloc[:,1] = label_encoder.fit_transform(dataset.iloc[:,1])
dataset.iloc[:,2] = label_encoder.fit_transform(dataset.iloc[:,2])
dataset.iloc[:,7] = label_encoder.fit_transform(dataset.iloc[:,7])
dataset.iloc[:,-1] = label_encoder.fit_transform(dataset.iloc[:,-1])

X = dataset.iloc[:,:-1]

y = dataset.iloc[:,-1]

X = X.values
y = y.values

# print(y)
print("dataset_size: ", y.size)
# X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2)

dataset_size:  2584


In [3]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X, y, cv=10)

print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


[0.62162162 0.81467181 0.92664093 0.93436293 0.93410853 0.93410853
 0.92635659 0.93410853 0.93410853 0.93410853]
Accuracy: 0.89 (+/- 0.19)


In [4]:
from sklearn.model_selection import cross_val_predict

y_pred = cross_val_predict(clf, X, y, cv=10)

from sklearn.metrics import confusion_matrix
print("confusion_matrix: ",confusion_matrix(y,y_pred))

from sklearn.metrics import precision_score,recall_score
print("precision: ",precision_score(y,y_pred))
print("recall:", recall_score(y,y_pred))

confusion_matrix:  [[2264  150]
 [ 136   34]]
precision:  0.18478260869565216
recall: 0.2


In [5]:
#oversampling
# from imblearn.over_sampling import RandomOverSampler
# ros = RandomOverSampler()
# X_over, y_over = ros.fit_resample(X, y)

from imblearn.over_sampling import SMOTE
ros = SMOTE()
X_over, y_over = ros.fit_resample(X, y)

print("dataset_size: ", y_over.size)

scores_over = cross_val_score(clf, X_over, y_over, cv=10)

print(scores_over)
print("Accuracy of Oversampling: %0.2f (+/- %0.2f)" % (scores_over.mean(), scores_over.std() * 2))

y_pred_over = cross_val_predict(clf, X_over, y_over, cv=10)

print("confusion matrix of oversampling: ",confusion_matrix(y_over,y_pred_over))

print("precision of oversampling: ",precision_score(y_over,y_pred_over))
print("recall of oversampling:", recall_score(y_over,y_pred_over))

Using TensorFlow backend.


dataset_size:  4828
[0.44214876 0.5661157  0.60950413 0.65082645 0.59543568 0.59958506
 0.62863071 0.60373444 0.60995851 0.62240664]
Accuracy of Oversampling: 0.59 (+/- 0.11)
confusion matrix of oversampling:  [[1144 1270]
 [ 696 1718]]
precision of oversampling:  0.5749665327978581
recall of oversampling: 0.7116818558409279


In [6]:
#undersampling
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids()
X_under, y_under = cc.fit_resample(X, y)

print("dataset_size: ", y_under.size)

scores_under = cross_val_score(clf, X_under, y_under, cv=10)

print(scores_under)
print("Accuracy of Undersampling: %0.2f (+/- %0.2f)" % (scores_under.mean(), scores_under.std() * 2))

y_pred_under = cross_val_predict(clf, X_under, y_under, cv=10)

print("confusion matrix of undersampling: ",confusion_matrix(y_under,y_pred_under))

print("precision of undersampling: ",precision_score(y_under,y_pred_under))
print("recall of undersampling:", recall_score(y_under,y_pred_under))

dataset_size:  340
[0.55882353 0.58823529 0.67647059 0.67647059 0.64705882 0.67647059
 0.64705882 0.61764706 0.61764706 0.5       ]
Accuracy of Undersampling: 0.62 (+/- 0.11)
confusion matrix of undersampling:  [[ 64 106]
 [ 23 147]]
precision of undersampling:  0.5810276679841897
recall of undersampling: 0.8647058823529412


In [7]:
#combine oversampling and undersampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN()
X_combine, y_combine = smote_enn.fit_resample(X, y)

print("dataset_size: ", y_combine.size)

scores_combine = cross_val_score(clf, X_combine, y_combine, cv=10)

print(scores_combine)
print("Accuracy of combining oversampling and undersampling: %0.2f (+/- %0.2f)" % (scores_combine.mean(), scores_combine.std() * 2))

y_pred_combine = cross_val_predict(clf, X_combine, y_combine, cv=10)

print("confusion matrix of combining oversampling and undersampling: ",confusion_matrix(y_combine,y_pred_combine))

print("precision of combining oversampling and undersampling: ",precision_score(y_combine,y_pred_combine))
print("recall of combining oversampling and undersampling:", recall_score(y_combine,y_pred_combine))

dataset_size:  3201
[0.5046729  0.67912773 0.80373832 0.8317757  0.734375   0.69375
 0.7        0.70846395 0.67711599 0.73354232]
Accuracy of combining oversampling and undersampling: 0.71 (+/- 0.17)
confusion matrix of combining oversampling and undersampling:  [[1132  325]
 [ 614 1130]]
precision of combining oversampling and undersampling:  0.7766323024054983
recall of combining oversampling and undersampling: 0.6479357798165137


In [8]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

clf_fs = ExtraTreesClassifier(n_estimators=50)
clf_fs = clf_fs.fit(X_combine, y_combine)
model_fs = SelectFromModel(clf_fs, prefit=True)
X_fs = model_fs.transform(X_combine)
print("feature dimension:", X_fs.shape[1])             

y_fs = y_combine

scores_fs = cross_val_score(clf, X_fs, y_fs, cv=10)

print(scores_fs)
print("Accuracy of feature selection: %0.2f (+/- %0.2f)" % (scores_fs.mean(), scores_fs.std() * 2))

y_pred_fs = cross_val_predict(clf, X_fs, y_fs, cv=10)

print("confusion matrix of feature selection: ",confusion_matrix(y_fs,y_pred_fs))

print("precision of feature selection: ",precision_score(y_fs,y_pred_fs))
print("recall of feature selection:", recall_score(y_fs,y_pred_fs))

feature dimension: 8
[0.5046729  0.67601246 0.68224299 0.70716511 0.646875   0.665625
 0.646875   0.64263323 0.62382445 0.68025078]
Accuracy of feature selection: 0.65 (+/- 0.11)
confusion matrix of feature selection:  [[1333  124]
 [1004  740]]
precision of feature selection:  0.8564814814814815
recall of feature selection: 0.4243119266055046


In [9]:
from sklearn.svm import LinearSVC

lsvc = LinearSVC(C=0.5, penalty="l1", dual=False).fit(X_combine, y_combine)
model_fs_2 = SelectFromModel(lsvc, prefit=True)
X_fs_2 = model_fs_2.transform(X_combine)
print("feature dimension:", X_fs_2.shape[1])
y_fs_2 = y_combine

scores_fs_2 = cross_val_score(clf, X_fs_2, y_fs_2, cv=10)

print(scores_fs_2)
print("Accuracy of feature selection: %0.2f (+/- %0.2f)" % (scores_fs_2.mean(), scores_fs_2.std() * 2))

y_pred_fs_2 = cross_val_predict(clf, X_fs_2, y_fs_2, cv=10)

print("confusion matrix of feature selection: ",confusion_matrix(y_fs_2,y_pred_fs_2))

print("precision of feature selection: ",precision_score(y_fs_2,y_pred_fs_2))
print("recall of feature selection:", recall_score(y_fs_2,y_pred_fs_2))

feature dimension: 14
[0.75077882 0.59813084 0.8411215  0.87227414 0.825      0.828125
 0.85       0.82131661 0.78056426 0.84952978]
Accuracy of feature selection: 0.80 (+/- 0.15)
confusion matrix of feature selection:  [[1251  206]
 [ 429 1315]]
precision of feature selection:  0.8645627876397107
recall of feature selection: 0.7540137614678899
